In [ ]:
import json
import pandas as pd
from textblob import TextBlob

def read_json(json_file: str)->list:
    """
    json file reader to open and read json files into a list
    Args:
    -----
    json_file: str - path of a json file
    
    Returns
    -------
    length of the json file and a list of json
    """
    
    tweets_data = []
    for tweets in open(json_file,'r'):
        tweets_data.append(json.loads(tweets))
    
    
    return len(tweets_data), tweets_data

class TweetDfExtractor:
    """
    this function will parse tweets json into a pandas dataframe
    
    Return
    ------
    dataframe
    """
    def __init__(self, tweets_list):
        
        self.tweets_list = tweets_list

    # an example function
    def find_statuses_count(self)->list:
        #create an empty list 
        statuses_count = list()
        
        #iterate through tweets list 
        for x in self.tweets_list:
            #append in created list statuses count
            statuses_count.append(x['user']['statuses_count'])
                                  
    def find_full_text(self)->list:
        
        # create an empty list
        text = []
        for tweets in self.tweets_list:
            if "retweeted_status" in [keys for keys,values in tweets.items()] and "extended_tweet" in tweets['retweeted_status'].keys():
                text.append(tweets['retweeted_status']['extended_tweet']['full_text'])
            else:
                text.append("Empty")
        return text
                                  
         
    def find_sentiments(self, text)->list:
        polarity = []
        subjectivity = []
        for tweets in text:
            blob = TextBlob(tweets)
            sentiment = blob.sentiment
            polarity.append(sentiment.polarity)
            subjectivity.append(sentiment.subjectivity)
        return polarity, subjectivity

    def find_created_time(self)->list:
        created_at = []
        for x in self.tweets_list:
            created_at.append(x['created_at'])
       
        return created_at

    def find_source(self)->list:
        # I can also use list comprehensiom
        source = [x['source'] for x in self.tweets_list]

        return source

    def find_screen_name(self)->list:
        screen_name = []
        for x in self.tweets_list:
            screen_name.append(x['user']['screen_name']) 
        return screen_name
                                  
    def find_followers_count(self)->list:
        followers_count =[]
        for x in self.tweets_list:
            followers_count.append(x['user']['followers_count'])
        return followers_count

    def find_friends_count(self)->list:
        friends_count= []
        for x in self.tweets_list:
            friends_count.append(x['user']['friends_count'])
        return friends_count

    def is_sensitive(self)->list:
        try:
            is_sensitive = []
            for tweets in self.tweets_list:
                if 'possibly_sensitive' in [keys for keys,values in tweets.items()]:
                    is_sensitive.append(tweets['possibly_sensitive'])
                else:
                    is_sensitive.append(0)
                    
        except KeyError:
            is_sensitive = None

        return is_sensitive

    def find_favourite_count(self)->list:
        favorite_count = list()
        for tweets in self.tweets_list:
            if 'retweeted_status' in [keys for keys,values in tweets.items()]:
                favorite_count.append(tweets['retweeted_status']['favorite_count'])
            else:
                favorite_count.append(0)
        return favorite_count
        
    
    def find_retweet_count(self)->list:
        retweet_count = list()
        for tweets in self.tweets_list:
            if 'retweeted_status' in [keys for keys,values in tweets.items()]:
                retweet_count.append(tweets['retweeted_status']['retweet_count'])
            else:
                retweet_count.append(0)
        return retweet_count

    def find_hashtags(self)->list:
        hashtags = []

        for tweets in self.tweets_list:
            hashtags.append(",".join([x['text'] for x in tweets['entities']['hashtags']]))
        return hashtags

    def find_mentions(self)->list:
        mentions = []
        for tweets in self.tweets_list:
            mentions.append( ", ".join([x['screen_name'] for x in tweets['entities']['user_mentions']]))

        return mentions
                                  


    def find_location(self)->list:
        try:
            location = self.tweets_list['user']['location']
        except TypeError:
            location = ''
        
        return location

    def find_lang(self)->list:
        lang = []
        for x in self.tweets_list:
            lang.append(x['lang'])
        return lang
        
        
    def get_tweet_df(self, save=False)->pd.DataFrame:
        """required column to be generated you should be creative and add more features"""
        
        columns = ['created_at', 'source', 'original_text','polarity','subjectivity', 'lang', 'favorite_count', 'retweet_count', 
            'original_author', 'followers_count','friends_count','possibly_sensitive', 'hashtags', 'user_mentions', 'place']
        
        created_at = self.find_created_time()
        source = self.find_source()
        text = self.find_full_text()
        polarity, subjectivity = self.find_sentiments(text)
        lang = self.find_lang()
        fav_count = self.find_favourite_count()
        retweet_count = self.find_retweet_count()
        screen_name = self.find_screen_name()
        follower_count = self.find_followers_count()
        friends_count = self.find_friends_count()
        sensitivity = self.is_sensitive()
        hashtags = self.find_hashtags()
        mentions = self.find_mentions()
        location = self.find_location()
        data = {"created_at":created_at,'source':source,'original_text':text,'polarity':polarity,'subjectivity':subjectivity,
                'lang':lang,'favorite_count':fav_count,'retweet_count':retweet_count,'original_author':screen_name, 
                'followers_count':follower_count,'friends_count':friends_count,'possibly_sensitive':sensitivity,
                'hashtags':hashtags,'user_mentions':mentions}

#         data = zip(created_at, source, text, polarity, subjectivity, lang, fav_count, retweet_count, screen_name, follower_count, friends_count, sensitivity, hashtags, mentions, location)
        df = pd.DataFrame(data, columns=columns)

        if save:
            df.to_csv('processed_tweet_data.csv', index=False)
            print('File Successfully Saved.!!!')
        
        return df

                
if __name__ == "__main__":
    # required column to be generated you should be creative and add more features
    columns = ['created_at', 'source', 'original_text','clean_text', 'sentiment','polarity','subjectivity', 'lang', 'favorite_count', 'retweet_count', 
    'original_author', 'screen_count', 'followers_count','friends_count','possibly_sensitive', 'hashtags', 'user_mentions', 'place', 'place_coord_boundaries']
    _, tweet_list = read_json("Economic_Twitter_Data.json")
    tweet = TweetDfExtractor(tweet_list)
    tweet_df = tweet.get_tweet_df() 

    # use all defined functions to generate a dataframe with the specified columns above

    

In [ ]:
class Clean_Tweets:
    """
    The PEP8 Standard AMAZING!!!
    """
    def __init__(self, df:pd.DataFrame):
        self.df = df
        print('Automation in Action...!!!')
        
    def drop_unwanted_column(self, df:pd.DataFrame)->pd.DataFrame:
        """
        remove rows that has column names. This error originated from
        the data collection stage.  
        """
        unwanted_rows = df[df['retweet_count'] == 'retweet_count' ].index
        df.drop(unwanted_rows , inplace=True)
        df = df[df['polarity'] != 'polarity']
        
        return df
    def drop_duplicate(self, df:pd.DataFrame)->pd.DataFrame:
        """
        drop duplicate rows
        """
        self.df = self.df.drop_duplicates().drop_duplicates(subset='original_text')
        
        return df
    def convert_to_datetime(self, df:pd.DataFrame)->pd.DataFrame:
        """
        convert column to datetime
        """
        self.df['created_at'] = pd.to_datetime(self.df['created_at'], errors='coerce')
        
        df = df[df['created_at'] >= '2020-12-31' ]
        
        return df
    
    def convert_to_numbers(self, df:pd.DataFrame)->pd.DataFrame:
        """
        convert columns like polarity, subjectivity, retweet_count
        favorite_count etc to numbers
        """
        
        # convert polarity column to numeric, coerce for setting Invalid parsing to nan
        df['polarity'] = pd.to_numeric(self.df['polarity'],errors="ignore")
        
        # same applies to other column
        df['subjectivity'] = pd.to_numeric(self.df['subjectivity'], errors="ignore")
        
        # putting square bracket while extracting column is the same putting a dot 
        df['favorite_count'] = pd.to_numeric( self.df.favorite_count, errors= "ignore")
        
        df['retweet_count'] = pd.to_numeric(self.df.retweet_count, errors="ignore")
        
        return df
    
    def remove_non_english_tweets(self, df:pd.DataFrame)->pd.DataFrame:
        """
        remove non english tweets from lang
        """
        # we use query function to query tweets whereby lang(language) = en(english)
        df = self.df.query("lang == 'en'")
        
        return df

In [ ]:
r = Clean_Tweets(tweet_df)

In [ ]:
df = r.remove_non_english_tweets(tweet_df)

In [ ]:
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')

In [ ]:
df = df[df['created_at'] >= '2020-12-31' ]

In [ ]:
df['polarity'] = pd.to_numeric(df['polarity'],errors="ignore")
        
# same applies to other column
df['subjectivity'] = pd.to_numeric(df['subjectivity'], errors="ignore")

# putting square bracket while extracting column is the same putting a dot 
df['favorite_count'] = pd.to_numeric( df.favorite_count, errors= "ignore")

df['retweet_count'] = pd.to_numeric(df.retweet_count, errors="ignore")


In [ ]:
df

# EDA

In [ ]:
a = df.corr()

In [ ]:
b = a.iloc[2:,2:]

In [ ]:
import seaborn as sns
sns.heatmap(b, annot = True)

In [ ]:
len(df['user_mentions'].unique())

In [ ]:
df.isnull().sum()

In [ ]:
df.drop("place",axis=1,inplace=True)

In [ ]:
columns = []
length = []
for i in df.columns:
    columns.append(i)
    length.append(len(df[i].unique()))

In [ ]:
dict_={}
r = []
for i,j in zip(columns,length):
    dict_[i] = j
    r.append(dict_)

In [ ]:
r_ = r[0]
r_

In [ ]:
r = pd.DataFrame(r_,index=[0])

In [ ]:
import matplotlib.pyplot as plt
sns.set()
r.plot(kind="bar")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),ncol=2)
plt.title("unique values for each column",fontweight="bold")
plt.xlabel("columns")
plt.ylabel("unique values");

In [ ]:
plt.scatter(df['favorite_count'],df['retweet_count'])
plt.title("favorite count against retweet count")
plt.xlabel("favorite count")
plt.ylabel("retweet count");

In [ ]:
import numpy as np
df['hashtags'] = df['hashtags'].replace('',np.NaN)

In [ ]:
a = list(df['hashtags'].unique())
a.remove(np.NaN)


In [ ]:
hashtags = []
values = []
for i in a:
    a = df[df['hashtags']==i]
    hashtags.append(i)
    values.append(len(a))
    

In [ ]:
d = []
D = {}
for i,j in zip(hashtags,values):
    D[i] = j
    d.append(D)

In [ ]:
w = d[0]

In [ ]:
sort_orders = sorted(w.items(), key=lambda x: x[1], reverse=True)

In [ ]:
e = []
t = []
for i in sort_orders:
    e.append(i[0])
    t.append(i[1])

In [ ]:
t[0:10]

In [ ]:
plt.bar(e[0:10],t[0:10])
plt.xticks(rotation=90)
plt.xlabel("hashtags")
plt.title("top 10 hashtags",fontweight="bold");

In [ ]:
v = list(df['original_author'].unique())

In [ ]:
authors = []
values_auth = []
for i in v:
    a = df[df['original_author']==i]
    authors.append(i)
    values_auth.append(len(a))

In [ ]:
p = []
t = {}
for i,j in zip(authors,values_auth):
    t[i] = j
    p.append(t)

In [ ]:
sort_orders_ = sorted(p[0].items(), key=lambda x: x[1], reverse=True)

In [ ]:
m = []
n = []
for i in sort_orders_:
    m.append(i[0])
    n.append(i[1])

In [ ]:
plt.bar(m[0:10],n[0:10])
plt.xticks(rotation=90)
plt.xlabel("original author")
plt.title("Top 10 authors");